In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.datasets import make_classification

In [2]:
# 1. Data - Generate random data
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

In [3]:
# 2. Preprocessing - Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [4]:
# 3. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [5]:
# 4. Create a Dataset for Contrastive Learning (using PyTorch)
class ContrastiveDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.long)

train_dataset = ContrastiveDataset(X_train, y_train)
test_dataset = ContrastiveDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [6]:
# 5. Define the Contrastive Model (simple neural network for representation learning)
class ContrastiveModel(nn.Module):
    def __init__(self):
        super(ContrastiveModel, self).__init__()
        self.fc1 = nn.Linear(20, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)  # Output 2 classes (for simplicity)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = ContrastiveModel()

In [7]:
# 6. Training the model using a Contrastive Loss Function
criterion = nn.CrossEntropyLoss()  # Using CrossEntropyLoss for simplicity
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

Epoch 1, Loss: 0.6457087351725652
Epoch 2, Loss: 0.5069469442734351
Epoch 3, Loss: 0.3887587395998148
Epoch 4, Loss: 0.3164394176923312
Epoch 5, Loss: 0.2977668276199928
Epoch 6, Loss: 0.2755122459851779
Epoch 7, Loss: 0.2661137546484287
Epoch 8, Loss: 0.26275987693896663
Epoch 9, Loss: 0.24906986837203687
Epoch 10, Loss: 0.23560507079729667


In [8]:
# 7. Evaluate the model
model.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.numpy())
        y_true.extend(labels.numpy())

accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy on test data: {accuracy:.4f}")

Accuracy on test data: 0.8300


In [9]:
# 8. Predict New Data (Using the trained model)
new_data = np.random.rand(5, 20)  # 5 new random data points
new_data_scaled = scaler.transform(new_data)  # Preprocessing step
new_data_tensor = torch.tensor(new_data_scaled, dtype=torch.float32)

model.eval()
with torch.no_grad():
    predictions = model(new_data_tensor)
    _, predicted_labels = torch.max(predictions, 1)

print(f"Predictions on new data: {predicted_labels.numpy()}")

Predictions on new data: [1 1 1 0 0]
